### First steps
We create some functions to parse the data and declare some constants that would be useful

In [2]:
# Constants
max_length_name = 20

# function to convert a string to a list of ascii values
def transform_string(string):
   string = string.lower()
   ascii_list = [ord(c) for c in string]
   list_ = ascii_list + [0] * (max_length_name - len(ascii_list))
   return np.array(list_).astype(np.float32)


### Load libraries/API's

In [3]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd


### Build Model

In [4]:
def build_model(vocab_size=255, embedding_dim=28, rnn_units=128):
    ''' Build a model with an embedding layer, bidirectional LSTM layer, and a dense layer.
        vocab_size: number of words in the vocabulary
        embedding_dim: dimension of the embedding vectors
        rnn_units: number of units in the RNN
    '''
    #build NN See. https://blog.paperspace.com/bidirectional-rnn-keras/
    #  https://towardsdatascience.com/boy-or-girl-a-machine-learning-web-app-to-detect-gender-from-name-16dc0331716c

    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=max_length_name))
    model.add(Bidirectional(LSTM(rnn_units, recurrent_dropout=0.2, dropout=0.2)))
    model.add(Dense(1, activation='sigmoid'))

    #Compile
    model.compile(loss='binary_crossentropy', optimizer=Adam(
        learning_rate=0.01), metrics=['accuracy'])

    return model


### Train the model

In [5]:
# load the data
df = pd.read_csv('unificado.csv')
print(df.shape)
df.head(10)

(12312, 2)


,name,sex
0,aaden,1
1,aaliyah,0
2,aamir,1
3,aarav,1
4,aaron,1
5,aaron,0
6,aaron,1
7,ab,1
8,abagael,0
9,abagail,0


In [6]:
model = build_model()

In [7]:

X = np.asarray(df['name'].values.tolist())
Y = np.asarray(df['sex'].values.tolist())

#Tranform names to numbers
X = np.array([transform_string(val) for val in X])

In [8]:
history = model.fit(x=X, y=Y, batch_size=500, epochs=50)
model.save('model_v1.h5')

Epoch 1/50
25/25 [==============================] - 30s 776ms/step - loss: 0.6647 - accuracy: 0.6018
Epoch 2/50
25/25 [==============================] - 20s 784ms/step - loss: 0.5860 - accuracy: 0.6892
Epoch 3/50
25/25 [==============================] - 16s 649ms/step - loss: 0.5296 - accuracy: 0.7412
Epoch 4/50
25/25 [==============================] - 16s 651ms/step - loss: 0.5108 - accuracy: 0.7560
Epoch 5/50
25/25 [==============================] - 22s 909ms/step - loss: 0.4892 - accuracy: 0.7667
Epoch 6/50
25/25 [==============================] - 20s 812ms/step - loss: 0.4692 - accuracy: 0.7765
Epoch 7/50
25/25 [==============================] - 20s 783ms/step - loss: 0.4629 - accuracy: 0.7776
Epoch 8/50
25/25 [==============================] - 20s 816ms/step - loss: 0.4517 - accuracy: 0.7872
Epoch 9/50
25/25 [==============================] - 21s 853ms/step - loss: 0.4491 - accuracy: 0.7876
Epoch 10/50
25/25 [==============================] - 21s 824ms/step - loss: 0.4415 - accura

In [9]:
#evaluate model
model.evaluate(X, Y)


385/385 [==============================] - 11s 24ms/step - loss: 0.1993 - accuracy: 0.9025


[0.19927044212818146, 0.9025341272354126]

In [10]:
def predict(name, model_loaded):
    data = np.array([transform_string(val) for val in name])
    result = model_loaded.predict(data)
    return ['m' if logit > 0.5 else 'f' for logit in result], [logit[0] for logit in result]


In [11]:
test_data = ['Roberto', 'Stephen', 'Allan', 'Amanda', 'beca']
predictions, logit = predict(test_data, model)
print(predictions)
print(logit)

1/1 [==============================] - 1s 968ms/step
['m', 'm', 'm', 'f', 'f']
[0.92509604, 0.96199495, 0.9767166, 0.18516563, 0.049329896]
